In [1]:
from config import *
import pandas as pd
import requests
import json
import time
from datetime import datetime

In [2]:
# underlying stock ticker symbol
ticker = 'SPY'

# expiration date of desired option where last number '0' is the days until expiration
exp_date = '2023-01-13:0'

# strike price of desired option in str
strike_price = '398.0'

# sets the desired net delta. net delta = number of share + option delta
target_net_delta = -2.0


# sets time interval (in seconds) for bot to check position and make adjustments in seconds
time_interval = 10

In [3]:
# Define TDAmeritrade Endpoint and Payload
tda_endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
tda_payload = {'apikey': TDA_ID,
               'symbol': ticker,
               'contractType': 'CALL',
               'includeQuotes': 'TRUE',
               'strategy': 'SINGLE',
               'expMonth': 'ALL',
               "isDelayed": 'true'}

# define alpaca endpoint and url
HEADERS = {'APCA-API-KEY-ID': ALPACA_KEY_ID, 'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY}
alpaca_endpoint = 'https://paper-api.alpaca.markets'
account_url = '{}/v2/account'.format(alpaca_endpoint)
orders_url = '{}/v2/orders'.format(alpaca_endpoint)
positions_url = '{}/v2/positions'.format(alpaca_endpoint)
portfolio_url = '{}/v2/account/portfolio/history'.format(alpaca_endpoint)

In [4]:
# this function will get the profit/loss on the stock position in alpaca
def get_pnl():
    data = {
        'period': '1D',
        'timeframe': '1D'
    }
    r = requests.get(portfolio_url, headers=HEADERS)
    return json.loads(r.content)


# returns the number of shares currently open
def num_of_shares(ticker):
    r = requests.get(positions_url + '/{}'.format(ticker), headers=HEADERS)
    return json.loads(r.content)


# puts in an order to buy/sell shares. side='buy' or 'sell'
def create_order(ticker, qty, side): 
    data = {
        'symbol': ticker,
        'qty': qty,
        'side': side,
        'type': 'market',
        'time_in_force': 'day'
    }
    r = requests.post(orders_url, json=data, headers=HEADERS)
    

# returns the delta of the short call
def get_call_delta(tda_data, exp_date, strike_price):
    return tda_data['callExpDateMap'][exp_date][strike_price][0]['delta'] * -100


# closes all alpaca positions
def close_all_positions():
    r = requests.delete(positions_url, headers=HEADERS)

In [5]:
# this returns how much you sold the option for in nominal dollars
r = requests.get(url=tda_endpoint, params=tda_payload)
tda_data = json.loads(r.content)
option_sell_price = tda_data['callExpDateMap'][exp_date][strike_price][0]['bid'] * 100

In [6]:
while (datetime.now().hour * 60 + datetime.now().minute) < 897: # bot keeps running until 3 minutes before market close, local time
    # gets the number of shares currently open 
    try:
        stock_delta = float(num_of_shares(ticker)['qty'])
    except:
        stock_delta = 0
    
    # gets the option delta and net_delta
    try:
        # get current call data
        r = requests.get(url=tda_endpoint, params=tda_payload)
        tda_data = json.loads(r.content)
        short_call_delta = get_call_delta(tda_data, exp_date, strike_price)
        
        # find net delta
        current_net_delta = round(stock_delta + short_call_delta, 0)
        
        # put in buy/sell orders based on net delta
        if current_net_delta < target_net_delta:
            create_order(ticker, (target_net_delta - current_net_delta), 'buy')
        elif current_net_delta > target_net_delta:
            create_order(ticker, (current_net_delta - target_net_delta), 'sell')
    except:
        print('error')
    
    option_pnl = round(option_sell_price - (tda_data['callExpDateMap'][exp_date][strike_price][0]['ask'] * 100), 2)
    stock_pnl = get_pnl()['profit_loss'][-1]
    
    print('open shares: ', stock_delta)
    print('option delta: ', round(short_call_delta, 1))
    print('net delta: ', current_net_delta)
    print('option profit/loss: ', option_pnl)
    print('stock profit/loss: ', stock_pnl)
    print('net profit/loss: ', round(stock_pnl + option_pnl, 2))
    print(datetime.now().replace(microsecond=0), '\n')
    print()
    
    time.sleep(time_interval) 

# after market closes, all alpaca positions will be closed
close_all_positions()

open shares:  0
option delta:  -33.7
net delta:  -34.0
option profit/loss:  -1.0
stock profit/loss:  0
net profit/loss:  -1.0
2023-01-13 13:20:10 


open shares:  32.0
option delta:  -33.8
net delta:  -2.0
option profit/loss:  -1.0
stock profit/loss:  -0.64
net profit/loss:  -1.64
2023-01-13 13:20:21 


open shares:  32.0
option delta:  -33.6
net delta:  -2.0
option profit/loss:  0.0
stock profit/loss:  -0.96
net profit/loss:  -0.96
2023-01-13 13:20:33 


open shares:  32.0
option delta:  -32.4
net delta:  -0.0
option profit/loss:  1.0
stock profit/loss:  -2.56
net profit/loss:  -1.56
2023-01-13 13:20:44 


open shares:  30.0
option delta:  -32.3
net delta:  -2.0
option profit/loss:  1.0
stock profit/loss:  -2.26
net profit/loss:  -1.26
2023-01-13 13:20:56 


open shares:  30.0
option delta:  -32.3
net delta:  -2.0
option profit/loss:  1.0
stock profit/loss:  -3.16
net profit/loss:  -2.16
2023-01-13 13:21:07 


open shares:  30.0
option delta:  -31.6
net delta:  -2.0
option profit/loss

open shares:  37.0
option delta:  -39.5
net delta:  -2.0
option profit/loss:  -8.0
stock profit/loss:  8.53
net profit/loss:  0.53
2023-01-13 13:30:36 


open shares:  37.0
option delta:  -40.1
net delta:  -3.0
option profit/loss:  -8.0
stock profit/loss:  8.53
net profit/loss:  0.53
2023-01-13 13:30:47 


open shares:  38.0
option delta:  -39.7
net delta:  -2.0
option profit/loss:  -7.0
stock profit/loss:  7.76
net profit/loss:  0.76
2023-01-13 13:30:59 


open shares:  38.0
option delta:  -40.8
net delta:  -3.0
option profit/loss:  -9.0
stock profit/loss:  10.8
net profit/loss:  1.8
2023-01-13 13:31:11 


open shares:  39.0
option delta:  -43.8
net delta:  -5.0
option profit/loss:  -12.0
stock profit/loss:  12.75
net profit/loss:  0.75
2023-01-13 13:31:23 


open shares:  42.0
option delta:  -41.9
net delta:  0.0
option profit/loss:  -10.0
stock profit/loss:  11.1
net profit/loss:  1.1
2023-01-13 13:31:34 


open shares:  40.0
option delta:  -42.9
net delta:  -3.0
option profit/loss:

open shares:  37.0
option delta:  -39.7
net delta:  -3.0
option profit/loss:  -6.0
stock profit/loss:  6.8
net profit/loss:  0.8
2023-01-13 13:41:08 


open shares:  38.0
option delta:  -39.3
net delta:  -1.0
option profit/loss:  -6.0
stock profit/loss:  6.79
net profit/loss:  0.79
2023-01-13 13:41:20 


open shares:  37.0
option delta:  -39.9
net delta:  -3.0
option profit/loss:  -7.0
stock profit/loss:  8.63
net profit/loss:  1.63
2023-01-13 13:41:31 


open shares:  38.0
option delta:  -39.5
net delta:  -2.0
option profit/loss:  -6.0
stock profit/loss:  5.97
net profit/loss:  -0.03
2023-01-13 13:41:43 


open shares:  38.0
option delta:  -36.7
net delta:  1.0
option profit/loss:  -2.0
stock profit/loss:  2.17
net profit/loss:  0.17
2023-01-13 13:41:54 


open shares:  35.0
option delta:  -36.9
net delta:  -2.0
option profit/loss:  -2.0
stock profit/loss:  2.87
net profit/loss:  0.87
2023-01-13 13:42:06 


open shares:  35.0
option delta:  -36.2
net delta:  -1.0
option profit/loss:  

open shares:  32.0
option delta:  -35.3
net delta:  -3.0
option profit/loss:  2.0
stock profit/loss:  -1.08
net profit/loss:  0.92
2023-01-13 13:51:34 


open shares:  33.0
option delta:  -34.4
net delta:  -1.0
option profit/loss:  3.0
stock profit/loss:  -2.08
net profit/loss:  0.92
2023-01-13 13:51:45 


open shares:  32.0
option delta:  -33.5
net delta:  -2.0
option profit/loss:  4.0
stock profit/loss:  -3.05
net profit/loss:  0.95
2023-01-13 13:51:57 


open shares:  32.0
option delta:  -34.2
net delta:  -2.0
option profit/loss:  3.0
stock profit/loss:  -2.09
net profit/loss:  0.91
2023-01-13 13:52:09 


open shares:  32.0
option delta:  -34.8
net delta:  -3.0
option profit/loss:  2.0
stock profit/loss:  -1.13
net profit/loss:  0.87
2023-01-13 13:52:20 


open shares:  33.0
option delta:  -35.2
net delta:  -2.0
option profit/loss:  2.0
stock profit/loss:  -0.8
net profit/loss:  1.2
2023-01-13 13:52:32 


open shares:  33.0
option delta:  -35.7
net delta:  -3.0
option profit/loss:  

open shares:  39.0
option delta:  -40.9
net delta:  -2.0
option profit/loss:  -5.0
stock profit/loss:  6.85
net profit/loss:  1.85
2023-01-13 14:02:04 


open shares:  39.0
option delta:  -43.2
net delta:  -4.0
option profit/loss:  -7.0
stock profit/loss:  8.02
net profit/loss:  1.02
2023-01-13 14:02:16 


open shares:  41.0
option delta:  -42.0
net delta:  -1.0
option profit/loss:  -5.0
stock profit/loss:  6.77
net profit/loss:  1.77
2023-01-13 14:02:28 


open shares:  40.0
option delta:  -42.0
net delta:  -2.0
option profit/loss:  -5.0
stock profit/loss:  5.95
net profit/loss:  0.95
2023-01-13 14:02:40 


open shares:  40.0
option delta:  -43.3
net delta:  -3.0
option profit/loss:  -7.0
stock profit/loss:  8.35
net profit/loss:  1.35
2023-01-13 14:02:52 


open shares:  41.0
option delta:  -44.3
net delta:  -3.0
option profit/loss:  -9.0
stock profit/loss:  11.21
net profit/loss:  2.21
2023-01-13 14:03:04 


open shares:  42.0
option delta:  -45.9
net delta:  -4.0
option profit/loss

open shares:  42.0
option delta:  -45.2
net delta:  -3.0
option profit/loss:  -8.0
stock profit/loss:  10.91
net profit/loss:  2.91
2023-01-13 14:12:25 


open shares:  43.0
option delta:  -46.0
net delta:  -3.0
option profit/loss:  -8.0
stock profit/loss:  8.77
net profit/loss:  0.77
2023-01-13 14:12:36 


open shares:  44.0
option delta:  -44.9
net delta:  -1.0
option profit/loss:  -7.0
stock profit/loss:  7.88
net profit/loss:  0.88
2023-01-13 14:12:48 


open shares:  43.0
option delta:  -46.1
net delta:  -3.0
option profit/loss:  -8.0
stock profit/loss:  9.16
net profit/loss:  1.16
2023-01-13 14:13:00 


open shares:  44.0
option delta:  -45.8
net delta:  -2.0
option profit/loss:  -7.0
stock profit/loss:  8.29
net profit/loss:  1.29
2023-01-13 14:13:11 


open shares:  44.0
option delta:  -46.3
net delta:  -2.0
option profit/loss:  -8.0
stock profit/loss:  10.49
net profit/loss:  2.49
2023-01-13 14:13:22 


open shares:  44.0
option delta:  -45.9
net delta:  -2.0
option profit/los

open shares:  48.0
option delta:  -52.3
net delta:  -4.0
option profit/loss:  -14.0
stock profit/loss:  16.91
net profit/loss:  2.91
2023-01-13 14:22:42 


open shares:  50.0
option delta:  -53.5
net delta:  -4.0
option profit/loss:  -15.0
stock profit/loss:  19.89
net profit/loss:  4.89
2023-01-13 14:22:54 


open shares:  52.0
option delta:  -54.9
net delta:  -3.0
option profit/loss:  -17.0
stock profit/loss:  19.89
net profit/loss:  2.89
2023-01-13 14:23:06 


open shares:  53.0
option delta:  -54.3
net delta:  -1.0
option profit/loss:  -18.0
stock profit/loss:  20.94
net profit/loss:  2.94
2023-01-13 14:23:18 


open shares:  52.0
option delta:  -55.7
net delta:  -4.0
option profit/loss:  -18.0
stock profit/loss:  21.97
net profit/loss:  3.97
2023-01-13 14:23:30 


open shares:  54.0
option delta:  -56.4
net delta:  -2.0
option profit/loss:  -13.0
stock profit/loss:  16.55
net profit/loss:  3.55
2023-01-13 14:23:42 


open shares:  54.0
option delta:  -51.6
net delta:  2.0
option p

open shares:  60.0
option delta:  -61.3
net delta:  -1.0
option profit/loss:  -19.0
stock profit/loss:  25.48
net profit/loss:  6.48
2023-01-13 14:33:02 


open shares:  59.0
option delta:  -63.2
net delta:  -4.0
option profit/loss:  -23.0
stock profit/loss:  29.61
net profit/loss:  6.61
2023-01-13 14:33:14 


open shares:  61.0
option delta:  -63.0
net delta:  -2.0
option profit/loss:  -23.0
stock profit/loss:  28.37
net profit/loss:  5.37
2023-01-13 14:33:25 


open shares:  61.0
option delta:  -64.1
net delta:  -3.0
option profit/loss:  -23.0
stock profit/loss:  29.59
net profit/loss:  6.59
2023-01-13 14:33:37 


open shares:  62.0
option delta:  -62.9
net delta:  -1.0
option profit/loss:  -21.0
stock profit/loss:  26.49
net profit/loss:  5.49
2023-01-13 14:33:49 


open shares:  61.0
option delta:  -63.1
net delta:  -2.0
option profit/loss:  -21.0
stock profit/loss:  26.48
net profit/loss:  5.48
2023-01-13 14:34:00 


open shares:  61.0
option delta:  -62.7
net delta:  -2.0
option 

open shares:  77.0
option delta:  -80.7
net delta:  -4.0
option profit/loss:  -69.0
stock profit/loss:  67.75
net profit/loss:  -1.25
2023-01-13 14:43:25 


open shares:  79.0
option delta:  -80.0
net delta:  -1.0
option profit/loss:  -64.0
stock profit/loss:  61.41
net profit/loss:  -2.59
2023-01-13 14:43:37 


open shares:  78.0
option delta:  -77.3
net delta:  1.0
option profit/loss:  -59.0
stock profit/loss:  52.82
net profit/loss:  -6.18
2023-01-13 14:43:48 


open shares:  75.0
option delta:  -75.0
net delta:  0.0
option profit/loss:  -55.0
stock profit/loss:  52.19
net profit/loss:  -2.81
2023-01-13 14:44:01 


open shares:  73.0
option delta:  -77.6
net delta:  -5.0
option profit/loss:  -63.0
stock profit/loss:  60.2
net profit/loss:  -2.8
2023-01-13 14:44:13 


open shares:  76.0
option delta:  -76.4
net delta:  -0.0
option profit/loss:  -57.0
stock profit/loss:  55.61
net profit/loss:  -1.39
2023-01-13 14:44:25 


open shares:  74.0
option delta:  -76.1
net delta:  -2.0
optio

open shares:  58.0
option delta:  -60.7
net delta:  -3.0
option profit/loss:  -8.0
stock profit/loss:  16.25
net profit/loss:  8.25
2023-01-13 14:53:50 


open shares:  59.0
option delta:  -62.9
net delta:  -4.0
option profit/loss:  -9.0
stock profit/loss:  16.84
net profit/loss:  7.84
2023-01-13 14:54:02 


open shares:  61.0
option delta:  -65.2
net delta:  -4.0
option profit/loss:  -13.0
stock profit/loss:  21.72
net profit/loss:  8.72
2023-01-13 14:54:13 


open shares:  63.0
option delta:  -66.4
net delta:  -3.0
option profit/loss:  -15.0
stock profit/loss:  18.57
net profit/loss:  3.57
2023-01-13 14:54:25 


open shares:  64.0
option delta:  -64.1
net delta:  -0.0
option profit/loss:  -10.0
stock profit/loss:  17.29
net profit/loss:  7.29
2023-01-13 14:54:37 


open shares:  62.0
option delta:  -73.1
net delta:  -11.0
option profit/loss:  -25.0
stock profit/loss:  30.93
net profit/loss:  5.93
2023-01-13 14:54:48 


open shares:  71.0
option delta:  -74.1
net delta:  -3.0
option p